In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import os
from glob import glob

import dmd
import grids as gr

In [ ]:
ds = gr.datasets.SD258
ds.load(layers=["orig"])

In [ ]:
gr.plot(ds)
print(len(ds))

In [ ]:
dmd_pipe = gr.pipelines.dmd()

out_dmd = dmd_pipe.run({"raw_image": ds[0]["orig"]})

print(out_dmd.keys())
print(out_dmd["dmd"].keys())
print(out_dmd["fingernet_output"]["minutiae"].shape, out_dmd["dmd"]["feature"].shape)

In [ ]:
from tqdm.notebook import tqdm
import pickle as pkl
import os
import torch

# Create template folder
template_folder = "dmd_templates"
os.makedirs(template_folder, exist_ok=True)
# Gallery folder
gallery_folder = os.path.join(template_folder, "gallery")
os.makedirs(gallery_folder, exist_ok=True)
# Query folder
query_folder = os.path.join(template_folder, "query")
os.makedirs(query_folder, exist_ok=True)

latents_template_list = []
latents = ds[{"sid": 0}]
references_template_list = []
references = ds[{"sid": 1}]

noext = lambda f: os.path.splitext(f)[0]

# Configuration
BATCH_SIZE = 64  # Process 64 images at once
USE_GPU = torch.cuda.is_available()
DEVICE = 'cuda' if USE_GPU else 'cpu'

print(f"🔧 Device: {DEVICE} | Batch size: {BATCH_SIZE}")

# Create GPU-accelerated DMD pipeline
mntstitch_model_path = "/fast8TB/jcontreras/work/grfinger/patchgrid/training_template/notebooks/mntstitch/models/resnet34_v2.pt"
dmd_pipe_gpu = gr.pipelines.dmd(mnt_extractor="mntstitch", mntstitch_model_path=mntstitch_model_path, device=DEVICE, use_gpu_patches=USE_GPU, max_batch_size=64)

# Helper function to process in batches
def process_batch(dataset, folder, batch_size=BATCH_SIZE):
    template_list = []
    
    for i in tqdm(range(0, len(dataset), batch_size), desc="Processing batches"):
        # Get batch
        batch_end = min(i + batch_size, len(dataset))
        batch_items = [dataset[j] for j in range(i, batch_end)]
        
        # Extract templates in batch
        for item in batch_items:
            out_dmd = dmd_pipe_gpu.run({"raw_image": item["orig"]})
            template = out_dmd["dmd"]
            template_list.append(template)
            
            # Save template
            template_path = os.path.join(folder, f"{noext(item.external_id)}.pkl")
            with open(template_path, "wb") as f:
                pkl.dump(template, f)
    
    return template_list

# Process latents (queries)
print(f"\n📦 Processing {len(latents)} query templates...")
latents_template_list = process_batch(latents, query_folder)

# Process references (gallery)
print(f"\n📦 Processing {len(references)} gallery templates...")
references_template_list = process_batch(references, gallery_folder)

print(f"\n✅ Done! Extracted {len(latents_template_list)} query and {len(references_template_list)} gallery templates")

## 🔍 Identification (1:N Matching)

Load the extracted templates from disk and perform 1:N identification to compute a similarity score matrix between all queries and gallery templates.

In [ ]:


# Option 1: Load from memory (if already extracted above)
if len(latents_template_list) > 0 and len(references_template_list) > 0:
    print("Using templates from memory...")
    query_templates = latents_template_list
    gallery_templates = references_template_list
else:
    # Option 2: Load from disk
    print("Loading templates from disk...")
    
    # Load query templates
    query_files = sorted(glob(os.path.join(query_folder, "*.pkl")))
    query_templates = []
    for f in query_files:
        with open(f, "rb") as pkl_file:
            query_templates.append(pkl.load(pkl_file))
    
    # Load gallery templates
    gallery_files = sorted(glob(os.path.join(gallery_folder, "*.pkl")))
    gallery_templates = []
    for f in gallery_files:
        with open(f, "rb") as pkl_file:
            gallery_templates.append(pkl.load(pkl_file))

print(f"📊 Loaded {len(query_templates)} query templates")
print(f"📊 Loaded {len(gallery_templates)} gallery templates")

# Create DMD matcher
matcher = dmd.DmdMatcher()

# Perform 1:N identification
print(f"\n🚀 Starting 1:N identification...")
print(f"   Computing {len(query_templates)} × {len(gallery_templates)} = {len(query_templates) * len(gallery_templates)} comparisons")

IDENTIFICATION_BATCH_SIZE = 256  # Batch size for identification
USE_GPU_MATCHING = torch.cuda.is_available()
MATCHING_DEVICE = 'cuda' if USE_GPU_MATCHING else 'cpu'

print(f"   Device: {MATCHING_DEVICE} | Batch size: {IDENTIFICATION_BATCH_SIZE}")

# Perform batch identification
scores_matrix = matcher.identify(
    query_templates, 
    gallery_templates, 
    device=MATCHING_DEVICE,
    batch_size=IDENTIFICATION_BATCH_SIZE
)

print(f"\n✅ Identification complete!")
print(f"   Score matrix shape: {scores_matrix.shape}")
print(f"   Score range: [{scores_matrix.min():.4f}, {scores_matrix.max():.4f}]")

# Create ground truth matrix (assuming same subject IDs)
# For SD258: subject IDs are the same for matching pairs
num_queries = len(query_templates)
num_gallery = len(gallery_templates)

# Assuming query subject 0 matches gallery subject 0, etc.
# Adjust this based on your dataset structure
target_matrix = np.zeros((num_queries, num_gallery), dtype=bool)
for i in range(min(num_queries, num_gallery)):
    target_matrix[i, i] = True  # Diagonal = genuine matches

print(f"\n📈 Ground truth matrix shape: {target_matrix.shape}")
print(f"   Genuine pairs: {target_matrix.sum()}")

## 📊 CMC Curve (Cumulative Match Characteristic)

Compute and plot the CMC curve to evaluate identification performance at different ranks.

In [ ]:


# Compute CMC curve
def compute_cmc(score_matrix, target_matrix, max_rank=20):
    """
    Compute Cumulative Match Characteristic (CMC) curve.
    
    Args:
        score_matrix: Score matrix [num_queries, num_gallery]
        target_matrix: Ground truth matrix [num_queries, num_gallery]
        max_rank: Maximum rank to compute (default: 20)
        
    Returns:
        cmc: CMC values for ranks 1 to max_rank
    """
    num_queries = score_matrix.shape[0]
    
    # Get sorted indices (highest scores first)
    sorted_indices = np.argsort(score_matrix, axis=1)[:, ::-1]
    
    # Initialize CMC
    cmc = np.zeros(max_rank)
    
    # For each rank
    for rank in range(max_rank):
        # Get top-(rank+1) predictions
        top_k_indices = sorted_indices[:, :rank+1]
        
        # Check if any of top-k matches is genuine
        correct = 0
        for q_idx in range(num_queries):
            top_k = top_k_indices[q_idx]
            if np.any(target_matrix[q_idx, top_k]):
                correct += 1
        
        cmc[rank] = correct / num_queries
    
    return cmc

# Compute CMC
print("Computing CMC curve...")
cmc = compute_cmc(scores_matrix, target_matrix, max_rank=257)

# Print key metrics
print(f"\n📈 Identification Results:")
print(f"   Rank-1:  {cmc[0]*100:.2f}%")
print(f"   Rank-5:  {cmc[4]*100:.2f}%")
print(f"   Rank-10: {cmc[9]*100:.2f}%")
print(f"   Rank-20: {cmc[19]*100:.2f}%")
print(f"   Rank-50: {cmc[49]*100:.2f}%")

# Plot CMC curve
plt.figure(figsize=(10, 6))
ranks = np.arange(1, len(cmc) + 1)
plt.plot(ranks, cmc * 100, 'b-', linewidth=2, markersize=6, label='DMD++')
plt.xlabel('Rank', fontsize=12)
plt.ylabel('Identification Rate (%)', fontsize=12)
plt.title('Cumulative Match Characteristic (CMC) Curve', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, linestyle='--')
plt.xlim([1, len(cmc)])
plt.ylim([0, 105])
plt.xticks(np.arange(1, len(cmc) + 1, 10))
plt.legend(fontsize=11)

# Add value annotations for key ranks
for rank_idx in [0, 4, 9, 19, 49, 99, 256]:  # Rank 1, 5, 10, 20
    if rank_idx < len(cmc):
        plt.annotate(f'{cmc[rank_idx]*100:.1f}%', 
                    xy=(rank_idx + 1, cmc[rank_idx] * 100),
                    xytext=(10, -10), textcoords='offset points',
                    fontsize=9, color='darkblue',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', color='darkblue'))

plt.tight_layout()
plt.show()

print("\n✅ CMC curve plotted!") # 86.4 at rank 1